In [ ]:
#Libraries for data manipulation
import pandas as pd
import numpy as np

#Library for metrics
from sklearn.metrics import roc_auc_score

#Library for splitting data
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('./forHiper')

In [ ]:
train.info()

In [ ]:
train = train.iloc[:, 4:]
x, y = train.iloc[:,1:], train.iloc[:,0]  #X tiene que tener todos los features distintos al target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

https://heartbeat.fritz.ai/hands-on-with-feature-selection-techniques-filter-methods-f248e0436ce5

https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(x_train, y_train)

feat_importances = pd.Series(model.feature_importances_, index = x_train.columns)
#feat_importances = feat_importances.nlargest(100)
feat_importances

## Constant Features

In [ ]:
# import and create the VarianceThreshold object.
from sklearn.feature_selection import VarianceThreshold
vs_constant = VarianceThreshold(threshold=0)

# fit the object to our data.
vs_constant.fit(x_train)

# get the constant colum names.
constant_columns = [column for column in x_train.columns
                    if column not in x_train.columns[vs_constant.get_support()]]

# detect constant categorical variables.
constant_cat_columns = [column for column in x_train.columns 
                        if (x_train[column].dtype == "O" and len(x_train[column].unique())  == 1 )]

# conctenating the two lists.
all_constant_columns = constant_cat_columns + constant_columns

# drop the constant columns
x_train = x_train.drop(labels=all_constant_columns, axis=1)
x_test = x_test.drop(labels=all_constant_columns, axis=1)

## Quasi-Constant Features

In [ ]:
# make a threshold for quasi constant.
threshold = 0.98

# create empty list
quasi_constant_feature = []

# loop over all the columns
for feature in x_train.columns:

    # calculate the ratio.
    predominant = (x_train[feature].value_counts() / np.float(len(x_train))).sort_values(ascending=False).values[0]
    
    # append the column name if it is bigger than the threshold
    if predominant >= threshold:
        quasi_constant_feature.append(feature)   

# drop the quasi constant columns
x_train = x_train.drop(labels=quasi_constant_feature, axis=1)
x_test = x_test.drop(labels=quasi_constant_feature, axis=1)

## Duplicated Features

In [ ]:
# transpose the feature matrice
train_features_T = x_train.T

# select the duplicated features columns names
duplicated_columns = train_features_T[train_features_T.duplicated()].index.values

# drop those columns
x_train = x_train.drop(labels=duplicated_columns, axis=1)
x_test = x_test.drop(labels=duplicated_columns, axis=1)

## Correlated Features

In [ ]:
# creating set to hold the correlated features
corr_features = set()

# create the correlation matrix (default to pearson)
corr_matrix = x_train.corr()

for i in range(len(corr_matrix .columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.8:
            colname = corr_matrix.columns[i]
            corr_features.add(colname)
            
x_train = x_train.drop(labels=corr_features, axis=1)
x_test = x_test.drop(labels=corr_features, axis=1)

## Feature importance

https://towardsdatascience.com/a-feature-selection-tool-for-machine-learning-in-python-b64dd23710f0

In [ ]:
#!pip install lightgbm
from feature_selector import FeatureSelector

# Features are in train and labels are in train_labels
fs = FeatureSelector(data = x_train, labels = y_train)

# Pass in the appropriate parameters
fs.identify_zero_importance(task = 'classification', 
                            eval_metric = 'auc', 
                            n_iterations = 10, 
                             early_stopping = True)

# plot the feature importances
fs.plot_feature_importances(threshold = 0.95, plot_n = 12)

1 features with zero importance after one-hot encoding.
270 features required for 0.95 of cumulative importance

In [ ]:
def getImportantFeatures():
    return list(fs.feature_importances.nlargest(270, 'importance').feature)